# Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how should it be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. User knows that he doesn't know the correct spelling OR he wants to get the results that follow some known pattern, so he uses so called wildcards - queries like 'retr*val';
2. User doesn't know the correct spelling OR he doesn't care OR he's in a rush OR he expects his mistakes will be corrected OR your option, so he makes mistakes and we need to handle them using:

    2.1. Simple spellchecker by Peter Norvig;
    
    2.2. Phonetic correction by means of Soundex algorithm;
    
    2.3. Trigrams with Jaccard coefficient.

## 1. Handling wildcards

We will handle wildcard queries using k-grams. K-grams is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf) *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

In [1]:
import urllib.request
data_url = "https://raw.githubusercontent.com/hsu-ai-course/hsu.ai/master/code/datasets/nlp/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))
        
print(*facts[-5:], sep='\n')

151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is about the same as all the humans.
155. On Jupiter and Saturn it rains diamonds.


In [2]:
import nltk
from collections import Counter
import re


def build_inverted_index_orig_forms(documents):
    #TODO build an inverted index of original word forms 
    # (without stemming, just word tokenized and lowercased)   
    inverted_index = {}
    for i, doc in enumerate(documents):
        tokens = nltk.word_tokenize(doc.lower())
        file_index = Counter(tokens)
        # update global index
        for term in file_index.keys():
            file_freq = file_index[term]
            if term not in inverted_index:                
                inverted_index[term] = [file_freq, (i, file_freq)]
            else:
                inverted_index[term][0] += file_freq
                inverted_index[term].append((i, file_freq))
    
    return inverted_index


def split_to_k_grams(word, k):
    return [word[i:i + k] for i in range(len(word) - k + 1)]


def build_k_gram_index(inverted_index, k):
    #TODO build index of k-grams for dictionary words. 
    # Padd with '$' ($word$) before splitting to k-grams    
    k_gram_index = {}
    for word in inverted_index.keys():
        augmented_w = '$' + word + '$'
        k_grams = split_to_k_grams(augmented_w, k)
        for gram in k_grams:
            if gram in k_gram_index:
                k_gram_index[gram].append(word)
            else:
                k_gram_index[gram] = [word]
    return k_gram_index


def generate_wildcard_options(wildcard, k_gram_index, inverted_index):
    #TODO for a given wildcard return all words matching it using k-grams
    # refer to book chapter 3.2.2
    # don't forget to pad wildcard with '$', when appropriate    
    augmented_wc = '$' + wildcard + '$'
    split_wc = augmented_wc.split("*")
    k_grams = set()
    k = len(list(k_gram_index.keys())[0])
    # forming list of k_grams for the query
    for split in split_wc:
        if len(split) >= k:
            k_grams.update(split_to_k_grams(split,k))
    sorted_postings = []
    # retrieve postings list for each k-gram
    for k_gram in k_grams:
        postings = set(k_gram_index[k_gram])
        sorted_postings.append((len(postings), postings))
    sorted_postings.sort()
    if not sorted_postings:
        return []
    # take the first (shortest) posting list and iteratively intersect with next ones
    corrections = set(sorted_postings[0][1])
    for i in range(1, len(sorted_postings)):
        corrections = set.intersection(corrections, sorted_postings[i][1])
    # remove not matching ones
    regexp = re.compile(wildcard.replace('*', '\w*'))
    filtered_corrections = []
    for correction in corrections:
        if re.search(regexp, correction) is not None:
            filtered_corrections.append(correction)
            
    sorted_filtered_corrections = sorted(filtered_corrections, key= lambda x: inverted_index.get(x)[0], reverse=True)
    return sorted_filtered_corrections


def search_wildcard(wildcard, k_gram_index, index, docs):
    #TODO retrive list of documnets (facts) that contain words matching wildcard
    wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index)
    doc_ids = set()
    for term in wildcard_options:        
        posting = index[term][1:]
        # extract document info only
        doc_ids.update([i[0] for i in posting])
    results = []
    for doc_id in doc_ids:
        results.append(docs[doc_id])
       
    return results


### 1.2 Tests

In [3]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "re*ed"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert(len(wildcard_options) >= 3)

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

assert "13. James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard("pres*dent", k_gram_index, index_orig_forms, facts)
assert "40. 9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index, index_orig_forms, facts)
assert "61. A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard("*price", k_gram_index, index_orig_forms, facts)

['reduced', 'received', 'recorded']
134. A person can live without food for about a month, but only about a week without water. If the amount of water in your body is reduced by just 1%, you’ll feel thirsty. If it’s reduced by 10%, you’ll die.
4. The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.
102. More than 50% of the people in the world have never made or received a telephone call.


## 2. Handling typos

### 2.1 Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [4]:
!pip install jsonlines
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [5]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

#### Build a dataset vocabulary
We will need it for Norvig's spellchecker as well as for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [6]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    words = nltk.word_tokenize(sent)    
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [7]:
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

58392

In [8]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6100), ('de', 80), ('deutsch', 4), ('nocomments', 2), ('you', 41999), ('can', 26004), ('disable', 527), ('comments', 351), ('for', 44674), ('the', 206912)]


### 2.2 Implement context-independent spellcheker ##

0) Write code to compute editorial distance

1) [Norvig's corrector](https://norvig.com/spell-correct.html)

2) [Soundex](https://en.wikipedia.org/wiki/Soundex)

3) Trigrams with Jaccard coefficient.

#### Editorial distance

Frequently used distance measure between two character sequences. We will use this distance to sort Soundex search results.

In [9]:
def edit_dist(s1, s2) -> int:
    # TODO compute the Damerau-Levenshtein distance between two given strings (s1 and s2)
    # from: https://www.guyrutenberg.com/2008/12/15/damerau-levenshtein-distance-in-python/
    
    xrange = range
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in xrange(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in xrange(-1,lenstr2+1):
        d[(-1,j)] = j+1
 
    for i in xrange(lenstr1):
        for j in xrange(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition
 
    return d[lenstr1-1,lenstr2-1]

#### Tests

In [10]:
assert edit_dist("korrectud", "corrected") == 2, "Edit distance is computed incorrectly"
assert edit_dist("soem", "some") == 1, "Edit distance is computed incorrectly"
assert edit_dist("one", "one") == 0, "Edit distance is computed incorrectly"

#### Norvig's spellchecker

In [11]:
WORDS = vocabulary

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def fix_typo_norvig(word) -> str:
    #TODO return best matching result for the word
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)


#### Tests

In [12]:
assert fix_typo_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_norvig("speling") == "spelling", "Norvig's correcter doesn't work"

#### Soundex 

For cases when the exact spelling is unknown, phonetic algorithms such as Soundex can be very helpful - they allow user to type a word the way he thinks it should sound, and then suggest the corrrect version. Go through *chapter 3.4* to understand how Soundex algorithm works.

In [13]:
def produce_soundex_code(word):
    #TODO implement Soundex algorithm, version from book chapter 3.4
    # input word is already lowercased
    # return Soundex 4-character code, like 'k450'
    
    if len(word) == 1:
        return word + "000"
    code = word[0]
    word = word[1:]
    groups = [['a', 'e', 'i', 'o', 'u', 'h', 'w', 'y'], # 0
              ['b', 'f', 'p', 'v'],  # 1
              ['c', 'g', 'j', 'k', 'q', 's', 'x', 'z'],  # 2
              ['d', 't'],  # 3
              ['l'],  # 4
              ['m', 'n'],  # 5
              ['r']]  # 6    
    # encoding all chars
    for num, group in enumerate(groups):
        for c in group:
            word = word.replace(c, str(num))
    # replacing consecutive identical digits
    word = re.sub(r'(.)\1{2,}', r'\1', word)
    # removing zeros
    word = word.replace('0', '')
    word += "000" # adding trailing zeros in case of short / empty word remaining
    code += word[:3]
    return code


def build_soundex_index(dictionary):
    #TODO build soundex index for dictionary words.
    # dictionary is a vocabulary of original words
    # output format: 'code1': ['word1_with_code1', 'word2_with_code1', ...]
    
    soundex_index = {}
    for word in dictionary.keys():
        code = produce_soundex_code(word)
        if code in soundex_index:
            soundex_index[code].append(word)
        else:
            soundex_index[code] = [word]
    return soundex_index


def fix_typo_soundex(word, soundex_index) -> list:
    #TODO return words from vocabulary that match with result by soundex fingerprint
    # ordered results by editorial distance
    code = produce_soundex_code(word)
    if code not in soundex_index:
        return [word]
    corrections = soundex_index[code]
    
    return sorted(corrections, key= lambda x: edit_dist(x, word))

#### Tests

In [14]:
soundex_index = build_soundex_index(vocabulary)

code1 = produce_soundex_code("britney")
code2 = produce_soundex_code("breatany")
print(code1, code2)
assert code1 == code2

print(fix_typo_soundex("enouhg", soundex_index))
assert "enough" in fix_typo_soundex("enouhg", soundex_index), "Assert soundex failed"

b635 b635
['enough', 'ensue', 'eng', 'enjoy', 'emoji', 'enqueue', 'ens', 'enc', 'emojii', 'enki', 'enso', 'enzo', 'enwiki', 'emesh', 'emg', 'emc', 'emas', 'euank', 'emac', 'emgo']


#### Trigrams with Jaccard coefficient

In [15]:
def intersection_over_union(set1, set2):
    return len(set1.intersection(set2)) / len(set1.union(set2))
    

def fix_typo_kgram(word, k_gram_index) -> list:
    #TODO return best matches with respect to Jaccard index
    k = len(list(k_gram_index.keys())[0])    
    augmented_word = '$' + word + '$'
    if len(augmented_word) < k:
        return None
       
    word_k_grams = set(split_to_k_grams(augmented_word,k))
    candidates = set()
    # retrieve postings list for each k-gram
    for k_gram in word_k_grams:
        candidates.update(k_gram_index.get(k_gram, []))
    # back to list
    candidates = list(candidates)
    
    if not candidates:
        return [word]
    
    return sorted(candidates, 
                  key = lambda x: intersection_over_union(set(split_to_k_grams('$' + x + '$',k)), word_k_grams),
                  reverse=True)

#### Tests

In [16]:
k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough', 'enought', 'eno', 'endogenous', 'enomem', 'enospc', 'enosys', 'enormous', 'renounce', 'en', 'exogenous', 'uh', 'enormously', 'homogenous', 'env', 'end', 'huh', 'hetrogenous', 'enh', 'ens']


### 2.3 Estimate quality

In [17]:
norvig, soundex, kgram = 0, 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, ss, sk = src.lower(), src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, ws, wk = fix_typo_norvig(word), \
                         fix_typo_soundex(word, soundex_index)[0], \
                         fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            ss = ss.replace(word, ws)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    soundex += int(ss == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"Soundex accuracy ({soundex}) = {soundex / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")

Norvig accuracy (2346) = 0.2346
Soundex accuracy (1673) = 0.1673
k-gram accuracy (1566) = 0.1566
